In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
from scipy.stats import hmean
import matplotlib.pyplot as plt
import os

In [2]:
def csv_parser(log_file, results_file):
    results_list = []
    with open(log_file,"r") as fp:
        for line in fp:
            split = line.split()
            split2 = line.split(",")
            if(len(split)>0):
                if("pmpakos@" in split[0]):
                    matrix_name = ''
#                     print("---------")

            if(len(split)>2):
                if((split[0]=="INFO:") & (split[1]=="loading") & (split[2]=="Mtx")):
                    print(split[-1])
                    matrix_name = split[-1].split("/")[-1]
                    matrix_name = matrix_name.replace(".mtx","")
            if(len(split2)>2):
                if(split2[1]!=" matrix"):
                    line=line.replace("app",matrix_name).strip("DATA_CSV:,")
#                     line=line.replace(",","\t")
                    results_list.append(line)
                    print(line)

    file = open(results_file,"w")
    for res in results_list:
        file.write(res)
    file.close()

In [21]:
def clean_matrix_generation_log(start_gen, results_gen):
    fw = open(results_gen,"w")
    fw_list = []
    cnt = 0
    with open(start_gen) as fp:
        for line in fp:
            if(">>>> " in line):
                if(line not in fw_list):
                    fw_list.append(line)
                    line = line.strip(">>>> ").replace(" ",",")#.replace(".mtx","")
                    line_spl = line.split(",")
                    line_spl[0] = line_spl[0].replace(".0_","_")
                    # line_spl[0] = line_spl[0].split("_")
                    # del line_spl[0][3]
                    # line_spl[0] = "_".join(line_spl[0])
                    line_new = ",".join(line_spl)
                    fw.write(line_new)
                else:
                    cnt+=1
    print("duplicates found :", cnt)
    fw.close()

def clean_results(start_list, results_list):
    for t1,t2 in zip(start_list,results_list):
        fw = open(t2,"w")
        with open(t1) as fp:
            for line in fp:
                if("DATA_CSV:," in line):
                    line_new = line.strip("DATA_CSV:,").replace(".0_","_")
                    fw.write(line_new)
        fw.close()
    
def read_results_single(result_file):
    header = ["matrix_name","original rows","original cols","original NNZs",
              "padded rows","padded cols","padded NNZs","padding overhead[%]",
              "num of runs","total run time[sec]","time[ms]/run","performance[GFLOPs]","performance (padded)[GFLOPs]","mem_footprint[MB]"]
    df = pd.read_table(result_file, delimiter =",", names=header)
    return df

def read_results(results_list, results_gen, results_csv):
    # this header was used for old result reporting
    # header = ["matrix_name","mem_range","avg_nnz_per_row","std_nnz_per_row","avg_bw","std_bw","avg_sc","std_sc"]
    # v2 dgal generator
    header = ["matrix_name", "distribution", "placement", "seed", 
              "nr_rows", "nr_cols", "nr_nnz", "density", "mem_footprint", "mem_range", 
              "avg_nnz_per_row", "std_nnz_per_row", 
              "avg_bw", "std_bw", "avg_bw_scaled", "std_bw_scaled", 
              "avg_sc", "std_sc", "avg_sc_scaled", "std_sc_scaled", "skew_coeff"]
    df_gen = pd.read_table(results_gen, delimiter =",", names=header) 
    
    dataframes = []
    for result_file in results_list:
        dataframes.append(read_results_single(result_file))
    matrices = dataframes[0]["matrix_name"].unique()

    n_runs = len(results_list)
    results_final = []
    cntt=0
    for matrix in matrices:
#         print(matrix)
        selected_gen = df_gen[df_gen["matrix_name"]==matrix]
        if(len(selected_gen)==0):
            print('\t', matrix)
            cntt+=1
        else:
            matrix_name = np.asarray(selected_gen["matrix_name"])[0]
            distribution = np.asarray(selected_gen["distribution"])[0]
            placement = np.asarray(selected_gen["placement"])[0]
            seed = np.asarray(selected_gen["seed"])[0]
            nr_rows = np.asarray(selected_gen["nr_rows"])[0]
            nr_cols = np.asarray(selected_gen["nr_cols"])[0]
            nr_nnz = np.asarray(selected_gen["nr_nnz"])[0]
            density = np.asarray(selected_gen["density"])[0]
            # if I want to show mem_footprint of CSR double representation
            mem_footprint = np.asarray(selected_gen["mem_footprint"])[0]
            
            mem_range = np.asarray(selected_gen["mem_range"])[0]
            avg_nnz_per_row = np.asarray(selected_gen["avg_nnz_per_row"])[0]
            std_nnz_per_row = np.asarray(selected_gen["std_nnz_per_row"])[0]
            avg_bw = np.asarray(selected_gen["avg_bw"])[0]
            std_bw = np.asarray(selected_gen["std_bw"])[0]
            avg_bw_scaled = np.asarray(selected_gen["avg_bw_scaled"])[0]
            std_bw_scaled = np.asarray(selected_gen["std_bw_scaled"])[0]
            avg_sc = np.asarray(selected_gen["avg_sc"])[0]
            std_sc = np.asarray(selected_gen["std_sc"])[0]
            avg_sc_scaled = np.asarray(selected_gen["avg_sc_scaled"])[0]
            std_sc_scaled = np.asarray(selected_gen["std_sc_scaled"])[0]
            skew_coeff = np.asarray(selected_gen["skew_coeff"])[0]

            matrix_list = matrix.split("_")
            mtx_name = matrix_list[0]

            # nr_rows = int(matrix_list[1])
            # nr_cols = int(matrix_list[2])

            # if(len(matrix_list)==7):#random
            #     placement = "random"
            #     diagonal_factor = 1
            # else: #diagonal
            #     placement = "diagonal"
            #     diagonal_factor = matrix_list[6]

            # seed = matrix_list[-1].strip("n").strip("g")

            # distr = list(filter(lambda x: x.isalpha(), matrix_list[-1]))[0]
            # if(distr=="g"):
            #     distribution="gamma"
            # else:
            #     distribution="normal"

            # nr_nnz = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original NNZs"])[0]
            # density = nr_nnz/(nr_rows*nr_cols)*100

            runtime_iter = np.zeros(shape=(n_runs,1))
            perf_padded = np.zeros(shape=(n_runs,1))
            # mem_footprint = np.zeros(shape=(n_runs,1))

            for run in range(0,n_runs):
                df = dataframes[run]
                runtime_iter[run] = np.asarray(df[df["matrix_name"]==matrix]["time[ms]/run"])[0]
                perf_padded[run] = np.asarray(df[df["matrix_name"]==matrix]["performance (padded)[GFLOPs]"])[0]
                # mem_footprint[run] = np.asarray(df[df["matrix_name"]==matrix]["mem_footprint[MB]"])[0]
            if(perf_padded[0][0]<0): # why? nobody knows why
                continue

            runtime_iter_hm = hmean(runtime_iter,axis=0)[0]
            perf_padded_hm = hmean(perf_padded,axis=0)[0]
            # if I want to show mem_footprint of Xilinx sparse double representation
            # mem_footprint = hmean(mem_footprint,axis=0)[0]

            # selected_gen = df_gen[df_gen["matrix_name"]==matrix]
            # mem_range = np.asarray(selected_gen["mem_range"])[0]
            # avg_nnz_per_row = np.asarray(selected_gen["avg_nnz_per_row"])[0]
            # std_nnz_per_row = np.asarray(selected_gen["std_nnz_per_row"])[0]
            # avg_bw = np.asarray(selected_gen["avg_bw"])[0]
            # std_bw = np.asarray(selected_gen["std_bw"])[0]
            # avg_sc = np.asarray(selected_gen["avg_sc"])[0]
            # std_sc = np.asarray(selected_gen["std_sc"])[0]/nr_cols

            W_avg = 33
            J_estimated = W_avg*runtime_iter_hm

            line_list = [matrix_name, distribution, placement, seed,
                         nr_rows, nr_cols, nr_nnz, density, mem_footprint, mem_range,
                         avg_nnz_per_row, std_nnz_per_row, 
                         avg_bw, std_bw, avg_bw_scaled, std_bw_scaled,
                         avg_sc, std_sc, avg_sc_scaled, std_sc_scaled,
                         skew_coeff,
                         "Xilinx_SpMV", runtime_iter_hm, perf_padded_hm, W_avg, J_estimated
                        ]
            line = ",".join(str(x) for x in line_list) + "\n"
            results_final.append(line)
        
    file = open(results_csv,"w")
    for line in results_final:
        file.write(line)
    file.close()
    print("Results for",len(results_final),"matrices for",results_csv.split("/")[-1])
    if(cntt>0):
        print(cntt, "matrices not found")


    
def extract_results_of_distr_memrange(distr_memrange):
    start_gen = "./generation_stats/"+distr_memrange+"_log.txt"
    results_gen = "./generation_stats/"+distr_memrange+"_log_CLEAN.txt"
    clean_matrix_generation_log(start_gen, results_gen)

    start_list = [
        "./dirty/"+distr_memrange+"_run1.txt",
    ]

    results_list = [
        "./clean/"+distr_memrange+"_run_CLEAN1.txt",
    ]
    clean_results(start_list, results_list)

    results_csv = "./results_"+distr_memrange+".csv"
    read_results(results_list, results_gen, results_csv)


In [6]:
%%time

distr_base = "normal"
mem_ranges = ["4-8","8-16","16-32","32-64","64-128","128-256","256-512","512-1024","1024-2048"]
# mem_ranges = ["1024-2048"]

for mem_range in mem_ranges:
    distr_memrange = distr_base + "_" + mem_range
    extract_results_of_distr_memrange(distr_memrange)

filenames = ["results_"+distr_base+"_"+mr+".csv" for mr in mem_ranges]

with open('../Benchmarks/xilinx_spmv_4-2048_normal_dataset_dtype-D.csv', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())

Results for 764 matrices for results_normal_4-8.csv
Results for 864 matrices for results_normal_8-16.csv
Results for 844 matrices for results_normal_16-32.csv
Results for 1212 matrices for results_normal_32-64.csv
Results for 1232 matrices for results_normal_64-128.csv
Results for 1415 matrices for results_normal_128-256.csv
Results for 908 matrices for results_normal_256-512.csv
Results for 968 matrices for results_normal_512-1024.csv
Results for 816 matrices for results_normal_1024-2048.csv
CPU times: user 24.7 s, sys: 122 ms, total: 24.8 s
Wall time: 26.7 s


In [20]:
%%time
    
distr_memrange = "validation_matrices_synthetic_small"
extract_results_of_distr_memrange(distr_memrange)
filename = "results_"+distr_memrange+".csv"

with open('../Benchmarks/xilinx_spmv_'+distr_memrange+'_dtype-D.csv', 'w') as outfile:
    with open(filename) as infile:
            outfile.write(infile.read())

duplicates found : 163
Results for 688 matrices for results_validation_matrices_synthetic_small.csv
CPU times: user 1.41 s, sys: 14.9 ms, total: 1.42 s
Wall time: 1.49 s


---

---

In [4]:
def read_results2(results_list, results_csv):
    dataframes = []
    for result_file in results_list:
        dataframes.append(read_results_single(result_file))
    # get matrices sorted by mem_footprint and then begin processing them (so that I do not have to sort them later...)
    matrices = dataframes[0]["matrix_name"].unique()
    # matrices = dataframes[0].groupby(["matrix name","mem_footprint[MB]"]).size().reset_index().sort_values("mem_footprint[MB]").reset_index(drop=True)["matrix name"]
    
    n_runs = len(results_list)
    results_final = []
    for matrix in matrices:
        nr_rows = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original rows"])[0]
        nr_cols = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original cols"])[0]        
        nr_nnz = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original NNZs"])[0]
        density = nr_nnz/(nr_rows*nr_cols)*100
        runtime_iter = np.zeros(shape=(n_runs,1))
        perf_padded = np.zeros(shape=(n_runs,1))
        mem_footprint = np.zeros(shape=(n_runs,1))

        for run in range(0,n_runs):
            df = dataframes[run]
            runtime_iter[run] = np.asarray(df[df["matrix_name"]==matrix]["time[ms]/run"])[0]
            perf_padded[run] = np.asarray(df[df["matrix_name"]==matrix]["performance (padded)[GFLOPs]"])[0]
            mem_footprint[run] = np.asarray(df[df["matrix_name"]==matrix]["mem_footprint[MB]"])[0]
        if(perf_padded[0][0]<0): # why? nobody knows why
            continue

        runtime_iter_hm = hmean(runtime_iter,axis=0)[0]
        perf_padded_hm = hmean(perf_padded,axis=0)[0]
        mem_footprint = hmean(mem_footprint,axis=0)[0]

        W_avg = 33
        J_estimated = W_avg*runtime_iter_hm
        
        line_list = [matrix, nr_rows, nr_cols, nr_nnz, density, mem_footprint, 
                     "Xilinx_SpMV", runtime_iter_hm, perf_padded_hm, W_avg, J_estimated
                    ]
        line = ",".join(str(x) for x in line_list) + "\n"
        results_final.append(line)

    file = open(results_csv,"w")
    for line in results_final:
        file.write(line)
    file.close()

In [16]:
start_list = [
    "./dirty/validation_matrices_run1.txt",
]

results_list = [
    "./clean/validation_matrices_run1.txt",
]
clean_results(start_list, results_list)

results_csv = "./results_validation_matrices.csv"
read_results2(results_list, results_csv)

---

In [5]:
def split_equally(starter_file, nummer):
    cnt_m = 0
    with open(starter_file) as fp:
        for line in fp:
            curr_m = int(line.split(" ")[0])
            cnt_m += curr_m
    step = cnt_m//nummer
    fp = open(starter_file,"r")
    lines = fp.readlines()
    cnt_file = 0
    for num in range(nummer):
        partial_file = starter_file.split(".txt")[0] + "_" + str(num+1) + ".txt"
        fw = open(partial_file,"w")
        cnt_m = 0
        while(cnt_m <= step and cnt_file < len(lines)):
            curr_line = lines[cnt_file]
            fw.write(curr_line)
            cnt_file += 1
            cnt_m += int(curr_line.split(" ")[0])
        fw.close()
        print(partial_file.split("/")[-1], "\t", num, "\t", cnt_file, "\t", cnt_m)
    fp.close()
    
starter_file = "../artificial_matrix_generation/pmpakos_impl/matrix_generation_parameters/double/validation_matrices_friends/dataset_small.txt"
nummer = 7

# split_equally(starter_file, nummer)


---

In [58]:
def clean_progress_log(start_gen, clean_gen, clean_gen2):
    fw = open(clean_gen,"w")
    # fw2 = open(clean_gen2,"w")
    cnt = 0
    with open(start_gen) as fp:
        for line in fp:
            # print(line)
            if(  ("%|" not in line) and not(line=="\n") and ("MEMORY USED :" not in line) and ("->" not in line)):
                fw.write(line)
            # if(  ("%|" not in line) and not(line=="\n") ):
                # fw2.write(line)
    fw.close()
    # fw2.close()
    
def clean_progress_log_wrapper(distr_memrange):
    prefix = "../artificial_matrix_generation/pmpakos_impl/matrix_generation_parameters/double/rest/progress"
    workers=0
    for ID in range(0,15,1):
        start_gen = prefix+distr_memrange+"_"+str(ID)
        if(os.path.exists(start_gen)):
            workers+=1
            clean_gen = prefix+distr_memrange+"_"+str(ID)+"_clean"
            clean_gen2 = prefix+distr_memrange+"_"+str(ID)+"_clean2"
            print(ID)
            clean_progress_log(start_gen, clean_gen, clean_gen2)
    if(workers>0):
        fw = open("../artificial_matrix_generation/pmpakos_impl/matrix_generation_parameters/double/rest/progress_total", "w")
        cnt = -workers
        for ID in range(0,15,1):
            start_gen = prefix+distr_memrange+"_"+str(ID)+"_clean"
            if(os.path.exists(start_gen)):
                with open(start_gen) as fp:
                    for line in fp:
                        fw.write(line)
                        if(">>>>" in line):
                            cnt+=1
        fw.close()
        total_set = "../artificial_matrix_generation/pmpakos_impl/matrix_generation_parameters/double/rest/dataset_small.txt"
        cnt_total=0
        if(os.path.exists(total_set)):
            cnt_total = sum(1 for line in open(total_set,"r"))
        print(cnt,"/",cnt_total,"ready.", cnt_total-cnt, "remain")


distr_memrange = ""
clean_progress_log_wrapper(distr_memrange)

1
2
3
4
5
6
7
8
9
10
11
12
13
0 / 13 ready. 13 remain
